In [16]:
from glob import glob
import pandas as pd
from tqdm import tnrange, tqdm_notebook

current = pd.read_csv('primaryMarketNotes_browseNotes_1-RETAIL.csv')

files = glob('LoanStats*.csv.zip')

def a_to_d_done(df):
    a_to_d = df['grade'].isin(['A', 'B', 'C', 'D'])
    done = df['loan_status'].isin(['Fully Paid', 'Charged Off'])
    return a_to_d & done

dfs = (pd.read_csv(file, skiprows=1, skipfooter=2, engine='python')
       .where(a_to_d_done)
       .dropna(subset=['grade', 'loan_status'])
       for file in tqdm_notebook(files))

df = pd.concat(dfs)

In [18]:
df.shape

(1283340, 144)

In [21]:
from sklearn.model_selection import train_test_split

historical, _ = train_test_split(df.copy(), train_size=0.30, 
                             stratify=df['loan_status'])

/Users/dwightchurchill/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [22]:
historical.to_csv('historical.csv', index=False)

In [23]:
historical.shape

(385002, 144)

In [25]:
common_cols = set(historical.columns) & set(current.columns)
just_historical = set(historical.columns) - set(current.columns)
just_current = set(current.columns) - set(historical.columns)

In [30]:
len(common_cols), len(just_historical), len(just_current)

(102, 42, 20)